In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load and prepare the datasets
data1 = pd.read_csv("../dataset/processed_data/Parkinson_CV_ensemble.tab", delimiter="\t")
data2 = pd.read_csv("../dataset/processed_data/Parkinson_FD_ensemble_pca.tab", delimiter="\t")

# Filter the last column as the label
label1 = data1.iloc[:, -1]
label2 = data2.iloc[:, -1]
label1 = label1.replace({'Co': 0, 'Pt': 1})
label2 = label2.replace({'Co': 0, 'Pt': 1})

# Remove the last column from the datasets
data1 = data1.iloc[:, :-1]
data2 = data2.iloc[:, :-1]

# Combine the datasets and labels
data = pd.concat([data1, data2], axis=1)
label = label1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

# Define the input shape
input_shape = (X_train.shape[1],)

# Define the attention mechanism layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# Define the model architecture with attention mechanism
def create_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    hidden = tf.keras.layers.Dense(32, activation='relu')(x)
    context_vector, attention_weights = Attention(32)(x, hidden)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(context_vector)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

# Instantiate the model
model = create_model(input_shape)

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=40, batch_size=8, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print('Test accuracy:', accuracy)



Epoch 1/40
27/27 [==============================] - 0s 4ms/step - loss: 102.3962 - accuracy: 0.6296 - val_loss: 109.5656 - val_accuracy: 0.4259
Epoch 2/40
27/27 [==============================] - 0s 1ms/step - loss: 115.3479 - accuracy: 0.5741 - val_loss: 136.4035 - val_accuracy: 0.4074
Epoch 3/40
27/27 [==============================] - 0s 1ms/step - loss: 108.4130 - accuracy: 0.5185 - val_loss: 108.7523 - val_accuracy: 0.4815
Epoch 4/40
27/27 [==============================] - 0s 1ms/step - loss: 102.5707 - accuracy: 0.6111 - val_loss: 116.1064 - val_accuracy: 0.3704
Epoch 5/40
27/27 [==============================] - 0s 1ms/step - loss: 78.8728 - accuracy: 0.5602 - val_loss: 101.3188 - val_accuracy: 0.4259
Epoch 6/40
27/27 [==============================] - 0s 1ms/step - loss: 81.1844 - accuracy: 0.5463 - val_loss: 142.7895 - val_accuracy: 0.3704
Epoch 7/40
27/27 [==============================] - 0s 1ms/step - loss: 80.9848 - accuracy: 0.5324 - val_loss: 117.8699 - val_accuracy: 0.